# Run easyocr

In [ ]:
from google.colab import drive
import os
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install easyocr fire lmdb opencv-python natsort nltk


In [ ]:
pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118


In [ ]:
%cd /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master_per

/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master_per


In [ ]:
!python3 create_lmdb_dataset.py \
  --inputPath output \
  --gtFile output/labels.txt \
  --outputPath lmdb_output

Created dataset with 18 samples


# Show LMDB

In [ ]:
import lmdb
import cv2
import numpy as np
from google.colab import drive
import os

In [ ]:

# مسیر دیتابیس LMDB
lmdb_path = "/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master_per/lmdb_output"

# باز کردن دیتابیس LMDB
env = lmdb.open(lmdb_path, readonly=True, lock=False)

# خواندن تعداد نمونه‌ها
with env.begin() as txn:
    num_samples = int(txn.get('num-samples'.encode()).decode())
    print(f"Number of samples: {num_samples}")

# خواندن تصاویر و برچسب‌ها
images = []
labels = []

with env.begin() as txn:
    for i in range(1, num_samples + 1):
        # کلیدهای تصویر و برچسب
        image_key = f'image-{i:09d}'.encode()
        label_key = f'label-{i:09d}'.encode()

        # خواندن داده باینری تصویر
        image_bin = txn.get(image_key)
        if image_bin is None:
            print(f"Image {image_key.decode()} not found")
            continue

        # خواندن برچسب
        label = txn.get(label_key).decode()
        if label is None:
            print(f"Label {label_key.decode()} not found")
            continue

        # تبدیل داده باینری به تصویر
        image_buf = np.frombuffer(image_bin, dtype=np.uint8)
        image = cv2.imdecode(image_buf, cv2.IMREAD_GRAYSCALE)
        if image is None:
            print(f"Failed to decode image {image_key.decode()}")
            continue

        images.append(image)
        labels.append(label)
        print(f"Image {i}: {label}")

# نمایش یا ذخیره تصاویر (اختیاری)
output_dir = "/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images"
os.makedirs(output_dir, exist_ok=True)

for i, (image, label) in enumerate(zip(images, labels)):
    output_path = os.path.join(output_dir, f"extracted_image_{i+1}.png")
    cv2.imwrite(output_path, image)
    print(f"Saved {output_path} with label: {label}")

# ذخیره برچسب‌ها در یک فایل (اختیاری)
with open("/content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_labels.txt", "w") as f:
    for i, label in enumerate(labels):
        f.write(f"image_{i+1}.png {label}\n")

print("Extraction completed!")

Number of samples: 18
Image 1: مخزن
Image 2: مخزن
Image 3: مخزن
Image 4: دستی
Image 5: دستی
Image 6: دستی
Image 7: هدف
Image 8: هدف
Image 9: هدف
Image 10: داد
Image 11: داد
Image 12: داد
Image 13: از
Image 14: از
Image 15: از
Image 16: مدیر
Image 17: مدیر
Image 18: مدیر
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_1.png with label: مخزن
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_2.png with label: مخزن
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_3.png with label: مخزن
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/deep-text-recognition-benchmark-master/show_er/extracted_images/extracted_image_4.png with label: دستی
Saved /content/drive/MyDrive/Ai_Lab/Hand_Writing/models_github/de

# Finr Tune

In [ ]:
!python3 train.py \
  --train_data  lmdb_output \
  --valid_data  lmdb_output \
  --select_data   "/" \
  --batch_ratio     1.0 \
  --Transformation    TPS \
  --FeatureExtraction    ResNet \
  --SequenceModeling    BiLSTM \
  --Prediction    Attn \
  --batch_size    2 \
  --data_filtering_off  \
  --workers    0 \
  --batch_max_length    80 \
  --num_iter    20 \
  --valInterval    5 \
  --saved_model   TPS-ResNet-BiLSTM-Attn.pth

--------------------------------------------------------------------------------
dataset_root: lmdb_output
opt.select_data: ['/']
opt.batch_ratio: ['1.0']
--------------------------------------------------------------------------------
dataset_root:    lmdb_output	 dataset: /
sub-directory:	/.	 num samples: 18
num total samples of /: 18 x 1.0 (total_data_usage_ratio) = 18
num samples of / per batch: 2 x 1.0 (batch_ratio) = 2
--------------------------------------------------------------------------------
Total_batch_size: 2 = 2
--------------------------------------------------------------------------------
dataset_root:    lmdb_output	 dataset: /
sub-directory:	/.	 num samples: 18
--------------------------------------------------------------------------------
model input parameters 32 100 20 1 512 256 71 80 TPS ResNet BiLSTM Attn
Skip Transformation.LocalizationNetwork.localization_fc2.weight as it is already initialized
Skip Transformation.LocalizationNetwork.localization_fc2.bias a

# Inference with Fine-tuned Model

In [ ]:
!python3 demo.py \
  --Transformation   TPS \
  --FeatureExtraction   ResNet \
  --SequenceModeling   BiLSTM \
  --Prediction   Attn \
  --image_folder   er/test_fa \
  --saved_model   saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth


model input parameters 32 100 20 1 512 256 38 25 TPS ResNet BiLSTM Attn
loading pretrained model from saved_models/TPS-ResNet-BiLSTM-Attn-Seed1111/best_accuracy.pth
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
--------------------------------------------------------------------------------
image_path               	predicted_labels         	confidence score
--------------------------------------------------------------------------------
er/test_fa/0.png         	ol                       	0.1018
er/test_fa/1.png         	tho                      	0.0802
er/test_fa/2.png         	gall 